In [34]:
# Titre : Transfert learning
# Importer les packages
import os
import numpy as np
import tensorflow as tf
from glob import glob
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Model
from joblib import load , dump

In [2]:
# Importer la base de données
data_path='/content/drive/MyDrive/Kaggle/pneumonia_data/chest_xray'

In [3]:
# Spécifier les chemins des bases de données train et test
train_dir=os.path.join(data_path,'train')
test_dir=os.path.join(data_path,'test') 

In [4]:
# Spécifier la taille des images
IMG_SHAPE=(224,224,3)
# Préparer le modèle de base
base_model=VGG16(input_shape=(IMG_SHAPE), include_top=False, weights='imagenet')
base_model.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [5]:
# Bloquer le modèle de base
base_model.trainable=False

In [6]:
# Taille de le sortie (7,7,512)
base_model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [24]:
# Ajouter les couches de sorties
x=Flatten()(base_model.output)
prediction=Dense(units=2, activation='softmax')(x)

In [25]:
# Création du modèle global
# Englober dans un seul modèle le petit et le grand modèle
model=Model(inputs=base_model.input , outputs= prediction)

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [27]:
# Compiler le modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Créer les photos augmentées
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
trainning_set=train_datagen.flow_from_directory(train_dir, target_size=(224,224),batch_size=32,class_mode='categorical')
test_set=test_datagen.flow_from_directory(test_dir, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [31]:
# Appliquer le modèle
model.fit_generator(trainning_set, epochs=5,validation_data=test_set)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
163/163 [==============================] - 1505s 9s/step - loss: 0.1729 - accuracy: 0.9331 - val_loss: 0.3947 - val_accuracy: 0.8734
Epoch 2/5
163/163 [==============================] - 139s 852ms/step - loss: 0.1050 - accuracy: 0.9595 - val_loss: 0.4248 - val_accuracy: 0.8798
Epoch 3/5
163/163 [==============================] - 137s 841ms/step - loss: 0.1079 - accuracy: 0.9586 - val_loss: 0.3113 - val_accuracy: 0.9071
Epoch 4/5
163/163 [==============================] - 137s 841ms/step - loss: 0.0856 - accuracy: 0.9645 - val_loss: 0.2844 - val_accuracy: 0.9183
Epoch 5/5
163/163 [==============================] - 136s 837ms/step - loss: 0.0844 - accuracy: 0.9668 - val_loss: 0.2483 - val_accuracy: 0.9151


In [32]:
# Précision du modèle
valid_loss,valid_accuracy=model.evaluate(test_set)

20/20 [==============================] - 9s 410ms/step - loss: 0.2483 - accuracy: 0.9151


In [33]:
print("Accuracy after tranfer learning : {}".format(valid_accuracy))

Accuracy after tranfer learning : 0.9150640964508057
